In [2]:
# !pip install stardist 
# !pip install numpy 
# !pip install matplotlib 
# !pip install opencv-python-headless 
# !pip install tensorflow 
# !pip install tqdm

In [3]:
import os
import numpy as np
import cv2
from stardist.models import StarDist2D
from stardist.plot import render_label
from matplotlib import pyplot as plt
from tqdm import tqdm
from skimage import exposure, morphology, filters, segmentation, measure
from scipy import ndimage as ndi
from skimage.feature import peak_local_max

In [4]:
def load_images_from_folder(folder):
    images = []
    for filename in tqdm(os.listdir(folder)):
        if filename.endswith(".tif"):
            img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                images.append(img)
    return images

folder_path = "original"
images = load_images_from_folder(folder_path)

100%|██████████| 28/28 [00:01<00:00, 21.41it/s]


In [5]:
model = StarDist2D.from_pretrained('2D_versatile_fluo')

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


In [6]:
def preprocess_image(img):
    blurred_img = cv2.GaussianBlur(img, (3, 3), 0)
    equalized_img = exposure.equalize_adapthist(blurred_img, clip_limit=0.03)
    background = filters.gaussian(equalized_img, sigma=10)
    subtracted_img = equalized_img - background
    edges = filters.sobel(subtracted_img)
    enhanced_img = np.clip(subtracted_img + edges, 0, 1)
    return enhanced_img

def segment_images(images, model):
    results = []
    for img in tqdm(images):
        preprocessed_img = preprocess_image(img)
        labels, details = model.predict_instances(preprocessed_img, n_tiles=(1, 1), prob_thresh=0.3, nms_thresh=0.05)
        
        cleaned_labels = morphology.remove_small_objects(labels, min_size=10)
        cleaned_labels = morphology.remove_small_holes(cleaned_labels, area_threshold=10)
        
        distance = ndi.distance_transform_edt(cleaned_labels)
        local_maxi = peak_local_max(distance, min_distance=2, threshold_abs=0.1, labels=cleaned_labels)
        
        markers = np.zeros_like(distance, dtype=bool)
        markers[tuple(local_maxi.T)] = True
        markers = measure.label(markers)
        
        watershed_labels = segmentation.watershed(-distance, markers, mask=cleaned_labels)
        
        refined_labels = morphology.dilation(watershed_labels, morphology.disk(1))
        
        results.append((img, refined_labels, details))
    return results

segmented_data = segment_images(images, model)

100%|██████████| 28/28 [00:27<00:00,  1.01it/s]


In [7]:
def visualize_and_save_results(data, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    for i, (img, labels, details) in enumerate(tqdm(data)):
        centroids = details['points']

        plt.figure(figsize=(16, 8), dpi=200)
        plt.subplot(1, 2, 1)
        plt.imshow(img, cmap='gray')
        plt.title('Original Image')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(render_label(labels, img=img))
        plt.title('Segmented Nuclei')
        plt.axis('off')

        plt.savefig(f"{save_dir}/segmented_{i}.png", bbox_inches='tight', pad_inches=0.1)
        plt.close()

visualize_and_save_results(segmented_data, "segmented")

100%|██████████| 28/28 [00:30<00:00,  1.10s/it]
